In [1]:
import numpy as np
import ipykernel as ipy
import pandas as pd
import yaml
import argparse
import typing
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score,recall_score,accuracy_score,precision_score,confusion_matrix,classification_report
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show, preserve, show_link, set_show_addr

In [9]:
df = pd.read_csv('C:\\Users\\ashis\\OneDrive\\Desktop\\MLOps\\churn_model\\data\\raw\\train.csv')
df.head()

,churn,number_vmail_messages,total_day_calls,total_eve_minutes,total_eve_charge,total_intl_minutes,number_customer_service_calls
0,no,26,123,195.5,16.62,13.7,1
1,no,0,114,121.2,10.30,12.2,0
2,no,0,71,61.9,5.26,6.6,2
3,no,0,113,148.3,12.61,10.1,3
4,no,24,88,348.5,29.62,7.5,3


In [12]:
train_x = df[[col for col in df.columns if col not in ['churn']]]
train_y = df['churn']

In [13]:
ebm = ExplainableBoostingClassifier()
ebm.fit(train_x, train_y)

ebm_global = ebm.explain_global(name = 'EBM')
   #preserve(ebm_global, 'number_customer_service_calls', 'number_customer_service_calls.html')
set_show_addr(("127.0.0.1", 5000))
show(ebm_global)

C:\Users\ashis\AppData\Local\Programs\Python\Python38\lib\site-packages\interpret\visual\udash.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\ashis\AppData\Local\Programs\Python\Python38\lib\site-packages\interpret\visual\udash.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\ashis\AppData\Local\Programs\Python\Python38\lib\site-packages\interpret\visual\udash.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


<!-- http://127.0.0.1:5000/2965392785904/ -->

In [15]:
ebm_local = ebm.explain_local(train_x[:10], train_y[:10], name='EBM')
show(ebm_local)

<!-- http://127.0.0.1:5000/2965088085856/ -->